# Making STAC items, collections, catalogs from a csv or dataframe

Thanks @axellblanc for several early iterations of this

In [214]:
import pandas as pd
import geopandas as gpd
from satstac import Catalog, Collection, Item
from shapely.geometry import box

In [333]:
df = pd.read_csv('data_znzfull.csv')
df.head()

,collection,prefix,id,datetime,td:title,td:description,td:type,td:classes,asset:raster:title,asset:raster:href,...,asset:vector:type,asset:thumbnail:title,asset:thumbnail:href,asset:thumbnail:type,asset:chips:title,asset:chips:href,asset:chips:type,asset:output_rst:title,asset:output_rst:href,asset:output_rst:type
0,zanzibar,znz001,znz001_buildings,2019-04-23T00:00:00Z,znz001 buildings,here is a description,segmentation,buildings,znz001_cog,https://oin-hotosm.s3.amazonaws.com/5afeda152b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,zanzibar,znz029,znz029_buildings,2019-04-23T00:00:00Z,znz029 buildings,here is a description,segmentation,buildings,znz029_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,znz029_thumbnail,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,image/png,znz029_chips,https://www.dropbox.com/sh/9ogls4r13zqfccl/AAD...,application/dropbox,model raster output,https://www.dropbox.com/s/o8je994h9uvibpu/znz0...,image/vnd.stac.geotiff; cloud-optimized=true
2,zanzibar,znz022,znz022_buildings,2019-04-23T00:00:00Z,znz022 buildings,here is a description,segmentation,buildings,znz022_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,zanzibar,znz023,znz023_buildings,2019-04-23T00:00:00Z,znz023 buildings,here is a description,segmentation,buildings,znz023_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,zanzibar,znz028,znz028_buildings,2019-04-23T00:00:00Z,znz028 buildings,here is a description,segmentation,buildings,znz028_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
collection,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,zanzibar,accra,accra
prefix,znz001,znz029,znz022,znz023,znz028,znz035,znz036,znz042,znz043,znz049,znz050,znz051,znz058,acc1,acc1
id,znz001_buildings,znz029_buildings,znz022_buildings,znz023_buildings,znz028_buildings,znz035_buildings,znz036_buildings,znz042_buildings,znz043_buildings,znz049_buildings,znz050_buildings,znz051_buildings,znz058_buildings,accra_1_buildings,accra_1_drains
datetime,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z,2019-04-23T00:00:00Z
td:title,znz001 buildings,znz029 buildings,znz022 buildings,znz023 buildings,znz028 buildings,znz035 buildings,znz036 buildings,znz042 buildings,znz043 buildings,znz049 buildings,znz050 buildings,znz051 buildings,znz058 buildings,accra 1 building labels,accra 1 drain labels
td:description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description,here is a description
td:type,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation,segmentation
td:classes,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,buildings,drains
asset:raster:title,znz001_cog,znz029_cog,znz022_cog,znz023_cog,znz028_cog,znz035_cog,znz036_cog,znz042_cog,znz043_cog,znz049_cog,znz050_cog,znz051_cog,znz058_cog,accra_1_cog,accra_1_cog
asset:raster:href,https://oin-hotosm.s3.amazonaws.com/5afeda152b...,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,https://oin-hotosm.s3.amazonaws.com/5b694a0f4b...,https://oin-hotosm.s3.amazonaws.com/5b694a0f4b...


In [335]:
def get_bbox(geojson_link):
    gdf = gpd.read_file(geojson_link)
    gdf = gdf[gdf['geometry'].isna() != True]
    return list(gdf['geometry'].total_bounds)

In [336]:
def get_geometry(address):
    data = gpd.read_file(address); 
    geom = []
    for f in data['geometry']:
        if(f is not None):
            #We now have to know wether it is a multipolygon or polygon or multiLineString or LineString,
            #in order to get the complete geometry and not only the first polygon/line
            if('polygon' in str(type(data['geometry'][0])) or 'polygon' in str(type(data['geometry'][1]))):   
                geom_type = 'Polygon'
                if('MultiPolygon' in str(type(data['geometry'][0]))):   
                    polygons = list(f)
                    for po in polygons:
                        x, y = po.exterior.coords.xy
                        geom.append([[[x[i],y[i]] for i in range(len(x))]])
                else:
                    x, y = f.exterior.coords.xy
                    geom.append([[[x[i],y[i]] for i in range(len(x))]])
            if('line' in str(type(data['geometry'][0])) or 'line' in str(type(data['geometry'][1]))):
                geom_type = 'LineString'
                if('Multi' in str(type(data['geometry'][0]))):   
                    lines = list(f)
                    for li in lines:
                        x, y = li.coords.xy
                        geom.append([[x[i],y[i]] for i in range(len(x))])
                else:
                    x, y = f.coords.xy
                    geom.append([[x[i],y[i]] for i in range(len(x))]) 
            if('point' in str(type(data['geometry'][0])) or 'point' in str(type(data['geometry'][1]))):
                geom_type = 'LineString'
                if('Multi' in str(type(data['geometry'][0]))):   
                    points = list(f)
                    for po in points:
                        x, y = li.coords.xy
                        geom.append([x[i],y[i]] for i in range(len(x)))
                else:
                    x, y = f.coords.xy
                    geom.append([x[i],y[i]] for i in range(len(x))) 
    geometry_item = {
        "type": "Multi" + geom_type,
        "coordinates": geom
        } 
    return geometry_item

In [337]:
properties_k = [o for o in list(df.columns) if 'datetime' in o or 'td:' in o]
assets_k = [o for o in list(df.columns) if 'asset:' in o]

In [338]:
properties_k, assets_k

(['datetime', 'td:title', 'td:description', 'td:type', 'td:classes'],
 ['asset:raster:title',
  'asset:raster:href',
  'asset:raster:type',
  'asset:vector:title',
  'asset:vector:href',
  'asset:vector:type',
  'asset:thumbnail:title',
  'asset:thumbnail:href',
  'asset:thumbnail:type',
  'asset:chips:title',
  'asset:chips:href',
  'asset:chips:type',
  'asset:output_rst:title',
  'asset:output_rst:href',
  'asset:output_rst:type'])

In [339]:
df.collection.unique()

array(['zanzibar', 'accra'], dtype=object)

In [340]:
collections_df = {o:df[df.collection==o] for o in df.collection.unique()}

In [341]:
collections_df['zanzibar']

,collection,prefix,id,datetime,td:title,td:description,td:type,td:classes,asset:raster:title,asset:raster:href,...,asset:vector:type,asset:thumbnail:title,asset:thumbnail:href,asset:thumbnail:type,asset:chips:title,asset:chips:href,asset:chips:type,asset:output_rst:title,asset:output_rst:href,asset:output_rst:type
0,zanzibar,znz001,znz001_buildings,2019-04-23T00:00:00Z,znz001 buildings,here is a description,segmentation,buildings,znz001_cog,https://oin-hotosm.s3.amazonaws.com/5afeda152b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,zanzibar,znz029,znz029_buildings,2019-04-23T00:00:00Z,znz029 buildings,here is a description,segmentation,buildings,znz029_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,znz029_thumbnail,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,image/png,znz029_chips,https://www.dropbox.com/sh/9ogls4r13zqfccl/AAD...,application/dropbox,model raster output,https://www.dropbox.com/s/o8je994h9uvibpu/znz0...,image/vnd.stac.geotiff; cloud-optimized=true
2,zanzibar,znz022,znz022_buildings,2019-04-23T00:00:00Z,znz022 buildings,here is a description,segmentation,buildings,znz022_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,zanzibar,znz023,znz023_buildings,2019-04-23T00:00:00Z,znz023 buildings,here is a description,segmentation,buildings,znz023_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,zanzibar,znz028,znz028_buildings,2019-04-23T00:00:00Z,znz028 buildings,here is a description,segmentation,buildings,znz028_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,zanzibar,znz035,znz035_buildings,2019-04-23T00:00:00Z,znz035 buildings,here is a description,segmentation,buildings,znz035_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,zanzibar,znz036,znz036_buildings,2019-04-23T00:00:00Z,znz036 buildings,here is a description,segmentation,buildings,znz036_cog,https://oin-hotosm.s3.amazonaws.com/5ae242fd0b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,zanzibar,znz042,znz042_buildings,2019-04-23T00:00:00Z,znz042 buildings,here is a description,segmentation,buildings,znz042_cog,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,zanzibar,znz043,znz043_buildings,2019-04-23T00:00:00Z,znz043 buildings,here is a description,segmentation,buildings,znz043_cog,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,zanzibar,znz049,znz049_buildings,2019-04-23T00:00:00Z,znz049 buildings,here is a description,segmentation,buildings,znz049_cog,https://oin-hotosm.s3.amazonaws.com/5ae318220b...,...,application/geo+json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
def make_items(df, convert_geom=True):
    items=[]

    for idx in range(len(df)):
        one_item = {}
        one_item['id'] = df.iloc[idx]['id']
        one_item['type'] = 'Feature'
        one_item['bbox'] = get_bbox(df.iloc[idx]['asset:vector:href'])
        
        # converting geometry of item.json to individual buildings if True, otherwise Polygon(bbox)
        if convert_geom: one_item['geometry'] = get_geometry(df.iloc[idx]['asset:vector:href'])
        else: 
            one_item['geometry'] = {
            "type": "Polygon",
            "coordinates": [[[x,y] for x,y in box(*one_item['bbox']).exterior.coords]]
        }
        
        # make dicts for each asset type
        one_item['assets'] = {}
        for a in assets_k:
            _, b, c = a.split(':')
            if not pd.isnull(df.iloc[idx][a]): one_item['assets'][b] = {}
        # update each asset
        for a in assets_k:
            _, b, c = a.split(':')
            if not pd.isnull(df.iloc[idx][a]): one_item['assets'][b].update({
                c : df.iloc[idx][a]
            })

        item_json = {}
        for p in properties_k: item_json[p] = df.iloc[idx][p]
        one_item['properties'] = item_json
        
        items.append(one_item)
    return items

In [343]:
znz_items = make_items(collections_df['zanzibar'], False)

In [344]:
znz_items[1]

{'id': 'znz029_buildings',
 'type': 'Feature',
 'bbox': [39.3411063109548,
  -5.878778696206506,
  39.356865475223195,
  -5.851576529338078],
 'geometry': {'type': 'Polygon',
  'coordinates': [[[39.356865475223195, -5.878778696206506],
    [39.356865475223195, -5.851576529338078],
    [39.3411063109548, -5.851576529338078],
    [39.3411063109548, -5.878778696206506],
    [39.356865475223195, -5.878778696206506]]]},
 'assets': {'raster': {'title': 'znz029_cog',
   'href': 'https://oin-hotosm.s3.amazonaws.com/5ae242fd0b093000130afd38/0/5ae242fd0b093000130afd39.tif',
   'type': 'image/vnd.stac.geotiff; cloud-optimized=true'},
  'vector': {'title': 'znz029_buildings_vector',
   'href': 'https://www.dropbox.com/sh/ct3s1x2a846x3yl/AADHytc8fSCf3gna0wNAW3lZa/grid_029.geojson?dl=1',
   'type': 'application/geo+json'},
  'thumbnail': {'title': 'znz029_thumbnail',
   'href': 'https://oin-hotosm.s3.amazonaws.com/5ae242fd0b093000130afd38/0/5ae242fd0b093000130afd39.png',
   'type': 'image/png'},
  '

In [345]:
def get_totalbounds(items):
    poly_list = []
    for item in items: poly_list.append(box(*item['bbox']))
    bbox_gpd = gpd.GeoSeries(poly_list)
    return list(bbox_gpd.total_bounds)

In [346]:
get_totalbounds(znz_items)

[39.28919876472999, -5.987213391948287, 39.36953783435138, -5.722212794937691]

# Catalog

In [353]:
stac_version = '0.7'

In [354]:
cat_json = {
    "id": "openml_data_demo",
    "title": "stac4ml demo",
    "description": "Sample ML training data in the STAC format",
    "stac_version": stac_version
}

mycat = Catalog(cat_json, root='https://my.cat')
mycat.save_as('catalog.json')

openml_data_demo

# Collections

In [355]:
coll_name = 'zanzibar'
items = make_items(collections_df[coll_name], True)

coll_json = {
    "stac_version": stac_version,
    "id": f'{coll_name}-collection',
    "title": f'{coll_name} AoI' ,
    "description": 'this description',
    "extent": {
        "spatial": get_totalbounds(items),
        "temporal": [
          "2019-05-23T00:00:00Z", None
        ]
      },
    
    "collection version": "1.0",
    "keywords": [
        "challenge",
        "zanzibar"
    ],
    "license": "CC-BY-4.0",
    }

coll = Collection(coll_json)
mycat.add_catalog(coll)

for i in items: coll.add_item(Item(i))

In [356]:
coll.data

{'stac_version': '0.7',
 'id': 'zanzibar-collection',
 'title': 'zanzibar AoI',
 'description': 'this description',
 'extent': {'spatial': [39.28919876472999,
   -5.987213391948287,
   39.36953783435138,
   -5.722212794937691],
  'temporal': ['2019-05-23T00:00:00Z', None]},
 'collection version': '1.0',
 'keywords': ['challenge', 'zanzibar'],
 'license': 'CC-BY-4.0',
 'links': [{'rel': 'self',
   'href': 'https://my.cat/zanzibar-collection/catalog.json'},
  {'rel': 'root', 'href': '../catalog.json'},
  {'rel': 'parent', 'href': '../catalog.json'}]}

In [357]:
coll_name = 'accra'

items = make_items(collections_df[coll_name])

coll_json = {
    "stac_version": stac_version,
    "id": f'{coll_name}-collection',
    "title": f'{coll_name} AoI' ,
    "description": 'this description',
    "extent": {
        "spatial": get_totalbounds(items),
        "temporal": [
          "2019-05-23T00:00:00Z", None
        ]
      },
    
    "collection version": "1.0",
    "keywords": [
        "challenge",
        "accra"
    ],
    "license": "CC-BY-4.0",
    }

coll = Collection(coll_json)
mycat.add_catalog(coll)

for i in items: coll.add_item(Item(i))

# Update root abs path, not needed in 0.7 anymore?

In [358]:
mycat.publish('https://zen-turing-2069dc.netlify.com')